In [ ]:
#####################
#     BY
# Isaac Gazimbe
####################

from flask import Flask
from flask_sqlalchemy import SQLAlchemy
import requests
import json

app = Flask(__name__)

app.config['SQLALCHEMY_DATABASE_URI']= "sqlite:///data.db"
db= SQLAlchemy(app)

#using object relational Maping
class Survivors(db.Model):
    sid= db.Column(db.Integer, primary_key=True)
    name= db.Column(db.String(80), unique=True, nullable=False )
    age= db.Column(db.Integer)
    gender= db.Column(db.String(80), unique=True, nullable=False )
    last_location= db.Column(db.String(120))
    resources= db.Column(db.String(120))
    flag=  db.Column(db.String(120))
    def __prep__(self):
        return f"{self.sid},{self.name},{self.age},{self.gender},{self.last_location},{self.resources},{self.flag}"
#create database
db.create_all()
#Add survivors
survivors = Survivors(name="Promise",age="25",gender="female", last_location="1523,-1452", resources="Food", flag="infected")
db.session.add(survivors)
db.session.commit()

#welcome endppoint
@app.route('/')
def index():
    return "Hello REST API"

#survivors endpoint
@app.route('/survivors')
def get_survivors():
    survivor= Survivors.query.all()
    output= []
    for survivor in survivors:
        survivor_data = {"name":survivor.name,"age":survivor.age,"gender":survivor.gender,"last_location":survivor.last_location,"resources":survivor.resources,"flag":survivor.flag}
        output.append(survivor_data)
    return {"survivors": output}

#single survivor endpoint
@app.route('/survivors/<id>')
def get_survivor(id):
    survivor = Survivor.query.get_or_404(id)
    return jsonfy({"name":survivor.name,"age":survivor.age,"gender":survivor.gender,"last_location":survivor.last_location,"resources":survivor.resources,"flag":survivor.flag})

#add survivors post method
@app.route('/survivors', methods=["POST"])
def add_survivor():
    survivor = Survivor(name=request.json["name"], age=request.json["age"], gender=request.json["gender"], last_location=request.json["last_location"], resources=request.json["resources"], flag=request.json["flag"])
    db.session.add(survivor)
    db.session.commit()
    return "Successfuly added"

#list of robots endpoint
@app.route('/robots')
def get_robots():
    #consuming an api
    response = requests.get('https://robotstakeover20210903110417.azurewebsites.net/robotcpu')
    #json format
    return {"robots":response.json()}

#percent Infected endpoint
@app.route('/percentInfected')
def percentInfected():
    infected = session.query(Survivors).filter(Survivors.flag.like('infected')).count()
    total= infected+session.query(Survivors).filter(Survivors.flag.like('not infected')).count() 
    percentInfected = (infected/total)*100
    return {"percentInfected": percentInfected}

#percent Not infected endpoint
@app.route('/percentNotInfected')
def percentNotInfected():
    notInfected = session.query(Survivors).filter(Survivors.flag.like('not infected')).count()
    total= NotInfected+session.query(Survivors).filter(Survivors.flag.like('infected')).count() 
    percentNotInfected = (infected/total)*100
    return {"percentNotInfected": percentNotInfected}

#infected list endpoint
@app.route('/infected')
def infected():
    infected = session.query(Survivors).filter(Survivors.flag.like('infected')).all()
    output= []
    for infect in infected:
        infected_data = {"name":survivor.name,"age":survivor.age,"gender":survivor.gender,"last_location":survivor.last_location,"resources":survivor.resources,"flag":survivor.flag}
        output.append(infected_data)
    return {"infected": output}

#not infected list endpoint
@app.route('/notInfected')
def notInfected():
    notInfected = session.query(Survivors).filter(Survivors.flag.like('not infected')).all()
    output= []
    for infect in notInfected:
        notInfected_data = {"name":survivor.name,"age":survivor.age,"gender":survivor.gender,"last_location":survivor.last_location,"resources":survivor.resources,"flag":survivor.flag}
        output.append(notInfected_data)
    return {"notInfected": output}
    

if __name__=="__main__":
    app.run()